In [1]:
import os
import numpy as np
# from torch.utils.data import DataLoader, Dataset

from preprocess import resize_input, train_test_split, read_raw
from ear_triplet import EarTriplet

import cv2
import torch


c:\Users\sebas\miniconda3\envs\trust-me\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_subjects = np.loadtxt("../train_subjects_mask.txt")

In [3]:
from PIL import Image
data_path = "../UERC"

In [4]:
ear_data = os.listdir(data_path)

ear_imgs = {}
for person in ear_data:
    if int(person) not in train_subjects:
        continue
    
    imgs = os.listdir("%s/%s" % (data_path, person))
    try:
        ear_imgs[person] = [
            cv2.cvtColor(
                np.asarray(Image.open(f"{data_path}/{person}/{img}")), cv2.COLOR_BGR2RGB
            )
            for img in imgs
        ]
    except Exception as e:
        print(e)

In [13]:
def split_triplets(X, y):
    # Constraint: anchor and positive must be the same person
    X, y = np.array(X), np.array(y)
    print("Classes: ", len(np.unique(y)))

    # Key has a list of indices of instances of that label
    # label_to_indices[label] returns indices of all the instances of class=label
    label_to_indices = {label: np.where(y == label)[0] for label in np.unique(y)}
    
    for key in label_to_indices.keys():
        np.random.shuffle(label_to_indices[key])

    an, pos, neg = [], [], []

    # For each class
    for i, label in enumerate(label_to_indices.keys()):
        subarray_size = len(label_to_indices[label]) // 2
        
        # Get random negative classes; negs = array of classes
        negs = list(label_to_indices.keys()).copy()
        negs.remove(label)
        negs = np.random.choice(negs, size=subarray_size, replace=False)
        while label in negs:
            negs = np.random.choice(list(label_to_indices.keys()), size=subarray_size, replace=False)

        # anchor and positive have label = label and negative is random
        an.extend(label_to_indices[label][:subarray_size])
        pos.extend(label_to_indices[label][subarray_size:][:subarray_size])
        
        # Negative = random class od vseh ostalih, vzami random instanco iz vsakega
        for neg_label in negs:
            neg.extend(np.random.choice(label_to_indices[neg_label], size=1))

    print(len(an), len(pos), len(neg))

    # Use the resulting arrays of indices to get the split arrays
    anchor_data, anchor_labels = X[an], y[an]
    positive_data, positive_labels = X[pos], y[pos]
    negative_data, negative_labels = X[neg], y[neg]
        
    return (anchor_data, anchor_labels), (positive_data, positive_labels), (negative_data, negative_labels)


In [6]:
X_train, X_eval, y_train, y_eval = train_test_split(ear_imgs)

In [7]:
len(X_train)

120809

In [ ]:
X_train = resize_input(X_train, 64, mode="train")

In [14]:

train_triplets = split_triplets(X_train, y_train)

train_anc, train_pos, train_neg = train_triplets

train_dataset = EarTriplet(train_anc[0], train_anc[1], train_pos[0], train_pos[1], train_neg[0], train_neg[1])

len(train_dataset.anchor_data), len(train_dataset.anchor_labels), len(train_dataset.positive_data), len(train_dataset.positive_labels), len(train_dataset.negative_data), len(train_dataset.negative_labels)


C:\Users\sebas\AppData\Local\Temp\ipykernel_23680\2878518757.py:3: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  X, y = np.array(X), np.array(y)
C:\Users\sebas\AppData\Local\Temp\ipykernel_23680\2878518757.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X, y = np.array(X), np.array(y)


Classes:  910
(909,) [1, 3, 4, 5, 6, 7, 8, 10, 12, 13]
(909,) [0, 3, 4, 5, 6, 7, 8, 10, 12, 13]
(909,) [0, 1, 4, 5, 6, 7, 8, 10, 12, 13]
(909,) [0, 1, 3, 5, 6, 7, 8, 10, 12, 13]
(909,) [0, 1, 3, 4, 6, 7, 8, 10, 12, 13]
(909,) [0, 1, 3, 4, 5, 7, 8, 10, 12, 13]
(909,) [0, 1, 3, 4, 5, 6, 8, 10, 12, 13]
(909,) [0, 1, 3, 4, 5, 6, 7, 10, 12, 13]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 12, 13]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 13]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4

(60149, 60149, 60149, 60149, 60149, 60149)

In [16]:
torch.save(train_dataset, "data/train_dataset.pt")

In [17]:
X_eval = resize_input(X_eval, 64, mode="test")

eval_triplets = split_triplets(X_eval, y_eval)

eval_anc, eval_pos, eval_neg = eval_triplets

eval_dataset = EarTriplet(eval_anc[0], eval_anc[1], eval_pos[0], eval_pos[1], eval_neg[0], eval_neg[1])


c:\Users\sebas\miniconda3\envs\trust-me\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(
c:\Users\sebas\miniconda3\envs\trust-me\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transfor

Classes:  910
(909,) [1, 3, 4, 5, 6, 7, 8, 10, 12, 13]
(909,) [0, 3, 4, 5, 6, 7, 8, 10, 12, 13]
(909,) [0, 1, 4, 5, 6, 7, 8, 10, 12, 13]
(909,) [0, 1, 3, 5, 6, 7, 8, 10, 12, 13]
(909,) [0, 1, 3, 4, 6, 7, 8, 10, 12, 13]
(909,) [0, 1, 3, 4, 5, 7, 8, 10, 12, 13]
(909,) [0, 1, 3, 4, 5, 6, 8, 10, 12, 13]
(909,) [0, 1, 3, 4, 5, 6, 7, 10, 12, 13]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 12, 13]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 13]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4, 5, 6, 7, 8, 10, 12]
(909,) [0, 1, 3, 4

In [18]:
torch.save(eval_dataset, "data/eval_dataset.pt")
